In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """


    N = 4
#     H = 0
#     for i in range(N):
#         H = H - qml.PauliZ(i) @ qml.PauliZ((i+1) % N)
    
#     for i in range(N):
#         H = H - h * qml.PauliX(i)

    coeffs = np.array([-1 for _ in range(N)] + [-h for _ in range(N)], requires_grad = False)
    obs = [qml.PauliZ(i) @ qml.PauliZ((i+1) % N) for i in range(N)] + [qml.PauliX(i) for i in range(N)]
    
    H = qml.Hamiltonian(coeffs, obs)
    
    return H


dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """


    p0 = params[0:3]
    p1 = params[3:6]
    p2 = params[6:9]
    p3 = params[9:]
    
    qml.Rot(p0[0], p0[1], p0[2], wires = 0)
    qml.Rot(p1[0], p1[1], p1[2], wires = 1)
    qml.Rot(p2[0], p2[1], p2[2], wires = 2)
    qml.Rot(p3[0], p3[1], p3[2], wires = 3)
    
    return qml.expval(H)


def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """


    N = 4
    
    params = np.random.normal(0, np.pi, N * 3, requires_grad = True)
    
    H = create_Hamiltonian(h)

    def cost(params):
        return model(params, H)
    
    opt = qml.AdamOptimizer(0.1)
    
    steps = 200
    for step in range(steps):
        params = opt.step(cost, params)
    
    return params


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")